In [98]:
import numpy as np 
import matplotlib.pyplot as plt
import scipy
from tqdm import tqdm
from sklearn.datasets import make_regression

# Convex Optimization - Homework 3

The goal is to : $\text{minimize } \frac{1}{2} || X w - y || _2 ^2 + \lambda || w || _2 ^2 $ (LASSO) 

We've seen in question 1 that the dual problem of LASSO is : $\text{minimize } \left[ v ^T Q v + p ^T v \right] \text{   subject to   } A v \preceq b$

## Question 2 : Implement the barrier method to solve QP.

#### 2.1 Write a function **$\text{Centering Step}(Q,p,A,b,t,v_0, \epsilon )$**

> This functions implements the **Newton method** to solve the *centering step* given the inputs $(Q, p, A, b)$, the *barrier method* parameter $t$ (see lectures), initial variable $v_0$ and a target precision $\epsilon$. The function outputs the sequence of variables iterates $(v_i)_{i=1,...,n_{\epsilon}}$, where nε is the number of iterations to obtain the ε precision. Use a backtracking line search with appropriate parameters.

In [58]:
# Define all the functions needed for the barrier method
def centering_step(Q, p, A, b, v0, t, epsilon, alpha, beta):
    """
    This function performs the centering step of the barrier method

    Parameters 
    ----------
    Q : numpy array
        Quadratic term of the target function
    p : numpy array
        Linear term of the target function
    A : numpy array
        Matrix of the constraints
    b : numpy array
        Vector of the constraints
    v0 : numpy array
        Initial point
    t : float
        Parameter of the barrier method
    epsilon : float
        Precision of the centering step
    alpha : float
        Parameter of the backtracking line search algorithm
    beta : float
        Parameter of the backtracking line search algorithm
    
    Returns 
    -------
    v : numpy array
        Suit of point of the centering step
    """

    ##### First we define some usefull functions for the centering step #####   
    def target_function(Q, p):
        """ 
        This function returns the target function of the quadratic programming problem
        """
        if not (b-A.dot(v)>0).all():
            raise Exception('Fail. Not feasible')
        f = lambda v : np.transpose(v) @ Q @ v + np.transpose(p) @ v
        return f

    def gradient_target(Q, p):
        """
        This function returns the gradient of the target function
        """
        grad = lambda v : 2 * np.dot(Q, v) + p
        return grad

    def hessian_target(Q):
        """
        This function returns the hessian of the target function
        """
        hess = lambda v : 2 * Q
        return hess

    def phi_function(A, b):
        """
        This function returns the phi function of the quadratic programming problem
        """
        A = np.array(A)
        phi = lambda v : - np.sum(np.log(- np.dot(A, v) - b))
        return phi

    def gradient_phi(A, b):
        """
        This function returns the gradient of the phi function
        """
        A = np.array(A)
        grad = lambda v : - np.dot(A.T, 1 / (np.dot(A, v) + b))
        return grad

    def hessian_phi(A, b):
        """
        This function returns the hessian of the phi function
        """
        A = np.array(A)
        hess = lambda v : np.sum(np.array([np.outer(A[i], A[i]) / (np.dot(A[i], v) - b[i]) ** 2 for i in range(len(A))]), axis=0)
        return hess

    def BM_function(f, phi):
        """
        This function returns the barrier method function
        """
        BM = lambda v, t : f(v) + t * phi(v)
        return BM

    def gradient_BM(grad_f, grad_phi, t):
        """
        This function returns the gradient of the barrier method function
        """
        grad_BM = lambda v : grad_f(v) + t * grad_phi(v)
        return grad_BM

    def hessian_BM(hess_f, hess_phi, t):
        """
        This function returns the hessian of the barrier method function
        """
        hess_BM = lambda v : hess_f(v) + t * hess_phi(v)
        return hess_BM
    

    ##### ----------------------------------------------------------------- #####

    # Initialize the centering step
    f = target_function(Q, p)
    grad_f = gradient_target(Q, p)
    hess_f = hessian_target(Q)
    phi = phi_function(A, b)
    grad_phi = gradient_phi(A, b)
    hess_phi = hessian_phi(A, b)
    complete_BM = BM_function(f, phi)
    BM = lambda v : complete_BM(v, t)
    grad_BM = gradient_BM(grad_f, grad_phi, t)
    hess_BM = hessian_BM(hess_f, hess_phi, t)

    # Then, we define the backtracking line search algorithm
    def backtrack_line_search(function, grad_function, v, delta):
        """ 
        This function implements the backtracking line search algorithm

        Parameters
        ----------
        function : function to minimize
        grad_function : gradient of the function to minimize
        v : current point
        alpha : parameter of the backtracking line search algorithm
        beta : parameter of the backtracking line search algorithm
        """
        eta = 1 
        print(f"Shape of v: {v.shape}")
        while function(v - eta * grad_function(v)) > function(v) - alpha * eta * np.dot(grad_function(v), grad_function(v)):
            eta *= beta 
        return eta
    
    # Finally, we implement the Newton's method
    liste_v = [v0]
    v = v0
    n_Nw_steps = 0
    
    while np.linalg.norm(grad_BM(v)) > epsilon:
         delta = - grad_BM(v)
         eta = backtrack_line_search(BM, grad_BM, v)
         v += eta * delta 
         liste_v.append(v)
         n_Nw_steps += 1
    return liste_v

In [104]:
# Define all the functions needed for the barrier method
def centering_step(Q, p, A, b, v0, t, epsilon, alpha, beta):
    """
    This function performs the centering step of the barrier method

    Parameters 
    ----------
    Q : numpy array
        Quadratic term of the target function
    p : numpy array
        Linear term of the target function
    A : numpy array
        Matrix of the constraints
    b : numpy array
        Vector of the constraints
    v0 : numpy array
        Initial point
    t : float
        Parameter of the barrier method
    epsilon : float
        Precision of the centering step
    alpha : float
        Parameter of the backtracking line search algorithm
    beta : float
        Parameter of the backtracking line search algorithm
    
    Returns 
    -------
    v : numpy array
        Suit of point of the centering step
    """
    ##### First we define some usefull functions for the centering step ##### 
    def f(Q, p, A, b, t, v):
        """
        Compute the value of the barrier objective function.

        Parameters:
        - Q : matrix Q in the quadratic term of the objective function
        - p : vector p in the linear term of the objective function
        - A : matrix A in the constraints
        - b : vector b in the constraints
        - t : current value of t (for barrier method)
        - v : current point

        Returns:
        - value of the objective function
        """
        if not (b - A.dot(v) > 0).all():
            raise Exception('Fail. Not feasible')
        return t * (v.T @ Q @ v + p.T.dot(v)) - np.sum(np.log(b - A.dot(v)))
     
    def grad(Q, p, A, b, t, v):
        """
        Compute the gradient of the barrier objective function.

        Parameters:
        - Q : matrix Q in the quadratic term of the objective function
        - p : vector p in the linear term of the objective function
        - A : matrix A in the constraints
        - b : vector b in the constraints
        - t : current value of t (for barrier method)
        - v : current point

        Returns:
        - gradient of the objective function
        """
        # Compute gradient of the objective function with respect to v
        barrier_grad = - A.T @ (1 / (b - A.dot(v)))
        return t * ((Q.T + Q).dot(v) + p) + barrier_grad


    def hessian(Q, p, A, b, t, v):
        """
        Compute the Hessian of the barrier objective function.

        Parameters:
        - Q : matrix Q in the quadratic term of the objective function
        - p : vector p in the linear term of the objective function
        - A : matrix A in the constraints
        - b : vector b in the constraints
        - t : current value of t (for barrier method)
        - v : current point

        Returns:
        - Hessian matrix of the objective function
        """
        temp = b - A.dot(v)
        hessian_matrix = 2 * t * Q  # Hessian of the quadratic part
        for i in range(A.shape[0]):
            # Add contribution of the barrier terms
            hessian_matrix += (1 / temp[i]**2) * A[i, :].reshape(-1, 1).dot(A[i, :].reshape(1, -1))
        return hessian_matrix
    ##### ----------------------------------------------------------------- #####

    def backtrack_line_search(Q, p, A, b, t, v, delta, alpha, beta):
        """
        Perform backtracking line search.

        Parameters:
        - Q : matrix Q in the quadratic term of the objective function
        - p : vector p in the linear term of the objective function
        - A : matrix A in the constraints
        - b : vector b in the constraints
        - t : current value of t (for barrier method)
        - v : current point
        - delta : descent direction (step to move along)
        - alpha : parameter for sufficient decrease condition
        - beta : parameter for step size reduction

        Returns:
        - optimal step size (rate)
        """
        rate = 1  # Initial step size
        while True:
            new_v = v + rate * delta
            # Check if the new point is feasible
            if not ((b - A.dot(new_v)) > 0).all():  
                rate *= beta  
                continue

            if f(Q, p, A, b, t, new_v) <= f(Q, p, A, b, t, v) + alpha * rate * np.dot(grad(Q, p, A, b, t, v).T, delta):
                break    
            rate *= beta  
        return rate
    
    # Finally, we implement the Newton's method
    liste_v = [v0]
    v = v0
    n_Nw_steps = 0
    
    while True:
         delta = - np.dot(scipy.linalg.inv(hessian(Q, p, A, b, t, v)), grad(Q, p, A, b, t, v))
         lambda_cond = -np.dot(grad(Q, p, A, b, t, v), (delta))
         if lambda_cond / 2 <= epsilon:
            break
         else:
            eta = backtrack_line_search(Q, p, A, b, t, v, delta, alpha, beta)
            v += eta * delta 
            liste_v.append(v)
            n_Nw_steps += 1
    return liste_v

#### 2.2 Write a function **$ \text{Barr Method}(Q, p, A, b, v_0, \epsilon )$**

> This functions implements the **barrier method** to solve QP using precedent function given the data inputs $(Q, p, A, b)$, a feasible point $v0 $, a precision criterion $\epsilon $. The function outputs the sequence of variables iterates $ (v_i) _{i = 1, ..., n_{\epsilon } } $, where $n_{\epsilon }$ is the number of iterations to obtain the $ \epsilon $ precision.

In [105]:
def Barrier_Method(Q, p, A, b, x0, eps, mu, alpha = 0.01, beta = 0.5, t = 1):
    """ 
    This function implements the barrier method
    
    Parameters
    ----------
    
    Q : numpy array
        Quadratic term of the target function
    p : numpy array
        Linear term of the target function
    A : numpy array
        Matrix of the constraints
    b : numpy array
        Vector of the constraints
    x0 : numpy array
        Initial point
    eps : float
        Precision of the barrier method
    mu : float
        Parameter of the barrier method
    alpha : float
        Parameter of the backtracking line search algorithm
    beta : float
        Parameter of the backtracking line search algorithm
    t : float   
        Parameter of the barrier method
    """
    m = len(b)    # Number of constraints
    liste_x = [x0]
    xt = x0
    n_centering_step = 0
    while True:
        xt = centering_step(Q, p, A, b, xt, t, eps, alpha, beta)
        n_centering_step += 1
        t *= mu
        liste_x.append(xt)
        if (m / t) < eps:
            break
    return liste_x

## Question 3: 

>Test your function on randomly generated matrices $X$ and observations $y$ with $ \lambda = 10 $. 

> Plot *precision criterion* and *gap* $ f( v_t)− f ^* $ in semilog scale (using the best value found for $f$ as a surrogate for $f ^* $). Repeat for different values of the barrier method parameter $ \mu = 2, 15, 50, 100, . . . $ and check the impact on $w$. 

> What would be an appropriate choice for $ \mu $ ?

In [106]:
np.random.seed(2024)
n = 10
d = 50
lamb = 10
X, y, coef = make_regression(n_samples=n, n_features=d, n_informative = 10, coef = True, noise = 1)
y = y.reshape((-1,1))
Q = (1 / 2) * np.eye(n) 
p = y
A = np.vstack([np.transpose(X), - np.transpose(X)])
b = lamb * np.ones((2 * d))
v0 = np.zeros((n,1))
eps = 10 ** -3

plt.figure(figsize = (12, 8))
for mu_param in tqdm([2, 15, 50, 100, 150]):
    optim_list  = Barrier_Method(Q, p, A, b, v0, eps, mu = mu_param, alpha = 0.01, beta = 0.5, t = 1)
    v_star = optim_list[-1]
    gap = [np.transpose(v) @ Q @ v + np.transpose(p) @ v - np.transpose(v_star) @ Q @ v_star + np.transpose(p) @ v_star for v in optim_list]
    plt.semilogy(optim_list, gap, label = f"mu = {mu_param}")

plt.xlabel("Values of v")
plt.ylabel("Gap")
plt.title("Gap between the optimal value and the value of the barrier method for different values of mu")   
plt.grid(True)
plt.show()
    

  0%|          | 0/5 [00:00<?, ?it/s]


ValueError: operands could not be broadcast together with shapes (100,) (10,10) 

<Figure size 1200x800 with 0 Axes>

## Considerations 

- Choice of $\mu $: if $\mu $ is too small, then many outer iterations might be needed; if $\mu $ is too big, then Newton’s method (each centering step) might take many iterations to converge.

- Choice of $t_0 $: if $t_0 $ is too small, then many outer iterations might be needed; if $t_0 $ is too big, then the first Newton’s solve (first centering step) might require many iterations to compute $x_0$. 

But, the performance of the barrier method is often quite robust to the choice of $\mu $ and $t_0 $ in practice.